In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
from simulation.simulation import parallelized_simulation_attack_state
from attack.attack import mad_fgsm, mad_pgd
from NeuralShield.AAAI21.models import loader
from NeuralShield.hyperparameter import benchmarks

In [3]:
import ray
_ = ray.init(num_cpus=50, num_gpus=1)

2020-11-05 12:07:28,649	INFO resource_spec.py:212 -- Starting Ray with 121.48 GiB memory available for workers and up to 56.07 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-11-05 12:07:28,774	WARNING services.py:923 -- Redis failed to start, retrying now.
2020-11-05 12:07:29,023	WARNING services.py:923 -- Redis failed to start, retrying now.
2020-11-05 12:07:29,437	INFO services.py:1165 -- View the Ray dashboard at localhost:8267


In [4]:
fgsm_res = {}
for env_name in benchmarks:
    if "BulletEnv" in env_name:
        for algo in benchmarks[env_name]["algos"]:
            if "a2c" in algo or "ppo" in algo:
                actor_net = loader.get_actor_net(env_name, algo)
                ret = parallelized_simulation(env_name, algo, actor_net, 1000,
                                              1000, mad_fgsm, 1, {"l_inf_norm": 5e-1}, thread_number=50)
                fgsm_res[f"{env_name}/{algo}"] = ret

(pid=12901) pybullet build time: Sep  4 2020 23:44:26
(pid=12941) pybullet build time: Sep  4 2020 23:44:26
(pid=12924) pybullet build time: Sep  4 2020 23:44:26
(pid=12911) pybullet build time: Sep  4 2020 23:44:26
(pid=13036) pybullet build time: Sep  4 2020 23:44:26
(pid=12946) pybullet build time: Sep  4 2020 23:44:26
(pid=13023) pybullet build time: Sep  4 2020 23:44:26
(pid=13021) pybullet build time: Sep  4 2020 23:44:26
(pid=12920) pybullet build time: Sep  4 2020 23:44:26
(pid=12907) pybullet build time: Sep  4 2020 23:44:26
(pid=12906) pybullet build time: Sep  4 2020 23:44:26
(pid=12913) pybullet build time: Sep  4 2020 23:44:26
(pid=12927) pybullet build time: Sep  4 2020 23:44:26
(pid=13232) pybullet build time: Sep  4 2020 23:44:26
(pid=12926) pybullet build time: Sep  4 2020 23:44:26
(pid=12944) pybullet build time: Sep  4 2020 23:44:26
(pid=12933) pybullet build time: Sep  4 2020 23:44:26
(pid=12939) pybullet build time: Sep  4 2020 23:44:26
(pid=12902) pybullet build t

In [5]:
baseline_res = {}
for env_name in benchmarks:
    if "BulletEnv" in env_name:
        for algo in benchmarks[env_name]["algos"]:
            if "a2c" in algo or "ppo" in algo:
                actor_net = loader.get_actor_net(env_name, algo)
                ret = parallelized_simulation(env_name, algo, actor_net, 1000,
                                              1000, mad_fgsm, 0, {"l_inf_norm": 5e-1}, thread_number=50)
                baseline_res[f"{env_name}/{algo}"] = ret

In [13]:
pgd_res = {}
for env_name in benchmarks:
    if "BulletEnv" in env_name:
        for algo in benchmarks[env_name]["algos"]:
            if "a2c" in algo or "ppo" in algo:
                actor_net = loader.get_actor_net(env_name, algo)
                ret = parallelized_simulation(env_name, algo, actor_net, 1000,
                                              1000, mad_pgd, 1, {"l_inf_norm": 5e-1, "lr": 1e-1}, thread_number=50)
                pgd_res[f"{env_name}/{algo}"] = ret

In [7]:
baseline_df = pd.DataFrame.from_dict(baseline_res, orient="index", columns=[
                                     "reward mean", "reward std", "unsafe states found"])
baseline_df.insert(3, "attack", None)
df_fgsm = pd.DataFrame.from_dict(fgsm_res, orient="index", columns=[
                                 "reward mean", "reward std", "unsafe states found"])
df_fgsm.insert(3, "attack", "MAD-FGSM")
df_pgd = pd.DataFrame.from_dict(pgd_res, orient="index", columns=[
                                "reward mean", "reward std", "unsafe states found"])
df_pgd.insert(3, "attack", "MAD-PGD")
df3 = pd.concat([baseline_df, df_fgsm, df_pgd])
df3.sort_index()